In [1]:
import pandas as pd
import numpy as np
from bert_serving.client import BertClient
bc = BertClient()
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Savannah\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWa

In [2]:
# Notes about Notations
#
#   [unclear]foo[unclear]
#     "foo" may be empty, contain part of the word(s), or a guess
#     Examples: 126,222,241,279,287,311
#
#   [unclear]
#     Missing end tag. I assume it's a placeholder for an unclear word
#     Examples:
#
#   [insertion]foo[insertion]
#     Word insertion
#     Examples: 51, 166
#
#   [/insertion]
#     Missing the first tag. I assume this is an error.
#     Examples: 57
#
#   [foo]
#      Word insertion
#      Examples: 28
#      Spelling correction/clarification (after the unmodified word)
#      Examples: 121,259, 280, 296
#   
#   (foo)
#      Spelling correction (after the unmodified word)
#      Examples: 312
#
#   [deletion]foo[/deletion]
#      "foo" may be empty. If so, I assume the removed word could not be interpreted
#      Examples: 184,242
#
#   [paragraph]
#      I assume this indicates a paragraph break
#      Examples: 179,236
#
#   [name of symbol]
#      Examples: 233
#
#   Take another look at the notations on lines 106,119,286,302,303
#
#   Not all spelling errors are noted. Some words between tags are also mispelled.

In [3]:
# Notes About Dataframe:
#   T1: Outfits, Multiple Choice (Question 63)
#   T2: Extra notes/adjustments to 63
#   T3: Comments about 63
#   T4: Comments about questionnaire (Question 64)
#   T5: ??
#   T7: ??
#   T8: ??
data_white = pd.read_excel('C:\\Users\\Savannah\\Documents\\Classes\\Summer2020\\Research\\ASWW2\\data\\Survey_32N_and_32W_consolidated.xlsx', sheet_name="Survey_32W")

# Notes About Dataframe:
#   T5: [What question is this?]
#   T6: ??
#   T8: ??
data_black = pd.read_excel('C:\\Users\\Savannah\\Documents\\Classes\\Summer2020\\Research\\ASWW2\\data\\Survey_32N_and_32W_consolidated.xlsx', sheet_name="Survey_32N")

# Drop unneeded columns
data_white.drop(columns=['Unnamed: 0', 'subject_id', 'image_name', 'image_name_2'], inplace=True)
data_black.drop(columns=['Unnamed: 0', 'subject_id', 'image_name', 'image_name_2'], inplace=True)

In [4]:
# Check out what's up with rows where the answer to 63 is 'nan'
display(data_white[data_white['T1'].isnull()])

# In some cases, multiple answers were selected.
#   (e.g. 'They should be in the same outfits' and 'It doesn't make any difference')
display(data_white.loc[data_white['T1'] == '[\'They should be in separate outfits\', "It doesn\'t make any difference"]'])
display(data_white.loc[data_white['T1'] == '[\'They should be together in the same outfits\', "It doesn\'t make any difference"]'])
display(data_white.loc[data_white['T1'] == "['They should be in separate outfits', 'They should be together in the same outfits']"])

#data_white.T1.unique()[8] #5,6,7,8
#temp = pd.DataFrame(data_white, columns=['T1'])

,T1,T2,T3,T4,T5,T7,T8
1860,NaN,NaN,NaN,If the government kept families on W.P.A. befo...,NaN,NaN,NaN
2015,NaN,NaN,They will be proud of their own outfit and try...,My answers pertaining to negroes are based on ...,They will be proud [unclear]if?in?[/unclear] t...,9,9
2312,NaN,NaN,Only in combat duty should they be in same out...,NaN,Only in combat duty should they be in same out...,9,9


,T1,T2,T3,T4,T5,T7,T8
1880,"['They should be in separate outfits', ""It doe...",NaN,NaN,This is nothing to do after retreat no place t...,NaN,12,NaN


,T1,T2,T3,T4,T5,T7,T8
1998,['They should be together in the same outfits'...,NA RW,NaN,Profanity has its place in the army - after al...,NaN,12,NaN


,T1,T2,T3,T4,T5,T7,T8
2310,"['They should be in separate outfits', 'They s...",NaN,yes,this is a hell place,NaN,NaN,NaN


In [12]:
# Example taken from line 267 of excel sheet from survey 32N

# Unmodified
original = "same times is verry hard when yau came aff a thirty miles [unclear]hike[/unclear] and yaur feets ar hurting. you go to the dactar and he gives yau a small bax of arspin to take. when ever you get ready to go some place yau have to get a pass and mast af the time they want give you and. after saldering hard frame manday marning until satureday noan. [paragraph] this is the reason i saw that negro shauld get a fair chance after the war is aver. the negroes aut hear go aut in the mauntain and sleep in the snaw and rain try to get ready far the battle he try to make things [unclear]easy[/unclear] far the peaples in civilans better far them."
original_embed = bc.encode([original])[0]

# Tags removed. Text between tags is kept.
no_tags = "same times is verry hard when yau came aff a thirty miles hike and yaur feets ar hurting. you go to the dactar and he gives yau a small bax of arspin to take. when ever you get ready to go some place yau have to get a pass and mast af the time they want give you and. after saldering hard frame manday marning until satureday noan. this is the reason i saw that negro shauld get a fair chance after the war is aver. the negroes aut hear go aut in the mauntain and sleep in the snaw and rain try to get ready far the battle he try to make things easy far the peaples in civilans better far them."
no_tags_embed = bc.encode([no_tags])[0]

# Tags removed along with text between
no_tagged_words = "same times is verry hard when yau came aff a thirty miles and yaur feets ar hurting. you go to the dactar and he gives yau a small bax of arspin to take. when ever you get ready to go some place yau have to get a pass and mast af the time they want give you and. after saldering hard frame manday marning until satureday noan. this is the reason i saw that negro shauld get a fair chance after the war is aver. the negroes aut hear go aut in the mauntain and sleep in the snaw and rain try to get ready far the battle he try to make things far the peaples in civilans better far them."
no_tagged_words_embed = bc.encode([no_tagged_words])[0]

# Tags removed, spelling corrected.
corrected = "sometimes it's very hard when you come off a thirty mile hike and your feet are hurting. you go to the doctor and he gives you a small box of aspirin to take. whenever you get ready to go someplace you have to get a pass and most of the time they won't give you and after saldering hard from monday morning until saturday noon. this is the reason i saw that negro should get a fair chance after the war is over. the negroes out here go out in the mountain and sleep in the snow and rain try to get ready for the battle he try to make things easy for the peoples in civilans better for them."
corrected_embed = bc.encode([corrected])[0]

print(np.dot(original_embed, no_tags_embed)/(np.linalg.norm(original_embed)*np.linalg.norm(no_tags_embed)))
print(np.dot(original_embed, no_tagged_words_embed)/(np.linalg.norm(original_embed)*np.linalg.norm(no_tagged_words_embed)))
print(np.dot(original_embed, corrected_embed)/(np.linalg.norm(original_embed)*np.linalg.norm(corrected_embed)))

print(np.dot(no_tags_embed, no_tagged_words_embed)/(np.linalg.norm(no_tags_embed)*np.linalg.norm(no_tagged_words_embed)))
print(np.dot(no_tags_embed, corrected_embed)/(np.linalg.norm(no_tags_embed)*np.linalg.norm(corrected_embed)))

print(np.dot(no_tagged_words_embed, corrected_embed)/(np.linalg.norm(no_tagged_words_embed)*np.linalg.norm(corrected_embed)))

0.9891761
0.98634607
0.84299785
0.99600714
0.85183334
0.83809066


In [ ]:
# TO DO: test out some spell-check libraries.